[View in Colaboratory](https://colab.research.google.com/github/NurikS/Q_learning/blob/master/Qlearning.ipynb)

# ***Q-learning with gym and numpy***
***In this project we will implement q - learning algorithm and try to train an agent to play a simple game in the gym environment.***
***First, we will need to import all the necessary libraries.***

In [1]:
!pip install gym # to install gym environment

    100% |████████████████████████████████| 1.5MB 6.7MB/s 
    100% |████████████████████████████████| 1.0MB 10.5MB/s 
  Running setup.py bdist_wheel for gym ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/cb/14/71/f4ab006b1e6ff75c2b54985c2f98d0644fffe9c1dddc670925
Successfully built gym


In [2]:
import numpy as np # for Q-learning
import gym # for our virtual environment
import random # to generate random numbers

***We are going to use a game called FrozenLake. The point of the game is to get safely to the goal without falling into the hole. There are 4 types of locations (S-start, F-frozen surface, H-hole, G-goal). Frozen surface - good, hole - bad. With this information we need to find the optimal way to reach the goal.***

***Let's create the environment.***

In [3]:
env = gym.make("FrozenLake-v0")

***Now, we need a Q table.***

In [21]:
state_size = env.observation_space.n
action_size = env.action_space.n

qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


***Now, we need to define our hyperparameters.***

In [26]:
tot_episodes = 10000      #total number of episodes
learning_rate = 0.8       #the rate of optimization
max_steps = 99            #max number of steps per episode
gamma = 0.98              #discount factor

#Exp-exp parameters
epsilon = 1.0             #probability to choose random action (exploration rate)
max_epsilon = 1.0         #exploration prob at start
min_epsilon = 0.01        #min exploration prob
epsilon_decay = 0.01      #exponential decay rate to choose random action


***Now, the fun part. Let's finally implement our Q-learning algorithm.***

In [27]:
rewards = [] # List of rewards.


for episode in range(tot_episodes):
  state = env.reset() # reset the environment.
  step = 0
  done = False
  tot_rewards = 0
  
  for step in range(max_steps):
    #Choose an action in a given state.
    #First, generate a random value.
    
    exp_exp_tradeoff = random.uniform(0,1)
    
    #If the random number is greater than the epsilon, choose the max value
    #from a Q table. Otherwise, choose a random action.
    
    if exp_exp_tradeoff > epsilon:
      action = np.argmax(qtable[state,:])
    else:
      action = env.action_space.sample()
    
    #Take the action and observe the new state and the reward.
    
    new_state, reward, done, info = env.step(action)
    
    #Based on those observations, update a Q table using the Bellman equation.
    #Q(s,a) => Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    
    qtable[state][action] += learning_rate*(reward + 
                             gamma*np.max(qtable[new_state, :])-qtable[state][action]) 
    tot_rewards += reward
    state = new_state
    
    if done == True:
      break
    
  episode += 1
  
  #Reduce epsilon, because, we need less and less exploration.
  
  epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-epsilon_decay*episode)
  rewards.append(tot_rewards)
  
print ("Score over time: " +  str(sum(rewards)/tot_episodes))
print(qtable)
    

Score over time: 0.4996
[[2.13763384e-01 2.08655724e-01 1.79022393e-01 1.81282644e-01]
 [5.70539896e-03 1.87495386e-02 1.04123776e-01 1.72364954e-01]
 [3.37433622e-02 5.51875265e-02 9.48302529e-02 1.33416497e-01]
 [1.87481682e-04 4.71427892e-02 2.23388281e-02 1.01636724e-01]
 [2.28978271e-01 1.92234836e-03 4.08337171e-02 7.91559764e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.60967862e-01 7.31037728e-09 8.79469873e-05 5.13036877e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.03101252e-02 2.58863628e-02 1.93751612e-02 3.30615523e-01]
 [2.60257221e-02 3.69015877e-01 2.80382572e-02 3.55246466e-02]
 [3.95769772e-01 5.87422112e-03 4.50768722e-03 6.45901101e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.42093446e-02 3.62407506e-02 8.14097084e-01 1.57047758e-01]
 [7.17992359e-01 7.55997114e-01 8.22157948e-01 2.12976364e-01]
 [0.00000000e+00 0.00000000e+00

***Now, let's apply our Q table to play the game.***

In [29]:
env.reset()

for episode in range(5):
  state = env.reset()
  step = 0
  done = False
  print("#########################")
  print("Episode", episode)
  
  for step in range(max_steps):
    env.render()
    action = np.argmax(qtable[state, :])
    new_state, reward, done, info = env.step(action)
    
    if done:
      break
    state = new_state
    
    
env.close()

#########################
('Episode', 0)

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
#########################
('Episode', 1)

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHF